In [1]:
# import libaries
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
import pyspark.sql.functions as f
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer



/usr/local/lib/python3.9/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/local/lib/python3.9/dist-packages/pandas/core/arrays/masked.py:64: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (


In [2]:
class PSTool:
    def __init__(self):
        print('Creating output folder')
        if os.path.exists('output'):
            pass
        else:
            os.makedirs('output')

    def pyspark_session(self, host_location):
        """
        Creates and returns spark session object
        """
        print('Starting session')
        sc = SparkContext(host_location)  # Create spark context
        spark = SparkSession(sc)  # Create session
        return spark

    def file_loader(self, path, delim, spark_obj, schema):
        print('Loading in file')
        data = spark_obj.read.options(delimiter=delim).option("header","False").csv(path, schema=schema)
        
        print('File loaded')
        return data

    def get_questions(self, df):
        pass

if __name__ == "__main__":
    pstool = PSTool()  # Instanciate object
    spk = pstool.pyspark_session('local[16]')  # start session
    # load data
    path = '/data/dataprocessing/interproscan/all_bacilli.tsv'
    # path = 'all_bacilli_subset.tsv'
    schema = StructType([
        StructField("Protein_accession", StringType(), True),
        StructField("Sequence_MD5_digest", StringType(), True),
        StructField("Sequence_length", IntegerType(), True),
        StructField("Analysis", StringType(), True),
        StructField("Signature_accession", StringType(), True),
        StructField("Signature_description", StringType(), True),
        StructField("Start_location", IntegerType(), True),
        StructField("Stop_location", IntegerType(), True),
        StructField("Score", FloatType(), True),
        StructField("Status", StringType(), True),
        StructField("Date", StringType(), True),
        StructField("InterPro_annotations_accession", StringType(), True),
        StructField("InterPro_annotations_description", StringType(), True),
        StructField("GO_annotations", StringType(), True),
        StructField("Pathways_annotations", StringType(), True)])
    
    df = pstool.file_loader(path, '\t', spk, schema)
#     pstool.get_questions(df)
#     print('Closing spark session')
#     spk.sparkContext.stop()
    df.printSchema()  # Shows column names and some info

Creating output folder
Starting session


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/10 22:23:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Loading in file
File loaded
root
 |-- Protein_accession: string (nullable = true)
 |-- Sequence_MD5_digest: string (nullable = true)
 |-- Sequence_length: integer (nullable = true)
 |-- Analysis: string (nullable = true)
 |-- Signature_accession: string (nullable = true)
 |-- Signature_description: string (nullable = true)
 |-- Start_location: integer (nullable = true)
 |-- Stop_location: integer (nullable = true)
 |-- Score: float (nullable = true)
 |-- Status: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- InterPro_annotations_accession: string (nullable = true)
 |-- InterPro_annotations_description: string (nullable = true)
 |-- GO_annotations: string (nullable = true)
 |-- Pathways_annotations: string (nullable = true)



### Data cleaning
The "function" of the protein which is the "class" your model should predict is defined as the InterPRO number which covers:
- ">" 90% of the protein's sequenc
- Covers the largest length of the sequence

As we are to train on the interpro number:
- Remove any rows without interpro number


In [3]:
# remove rows that do not have Interpro number
print(df.select('InterPro_annotations_accession').distinct().count())
IPRO_filt = df.filter(df["InterPro_annotations_accession"] != '-')
print(IPRO_filt.select('InterPro_annotations_accession').distinct().count())

# check amount of rows left. 
print('len:', len(df.columns))
print('count:' , df.count())
# check if the columns are propperly loaded in. 
df_sizes = IPRO_filt.withColumn('perc', abs(df.Start_location - df.Stop_location) / df.Sequence_length).sort('perc')
for i in df_sizes.columns:
    print(df_sizes.select(i).show(5))

9704


9703
len: 15


count: 4200591


+--------------------+
|   Protein_accession|
+--------------------+
|gi|510143242|gb|A...|
|gi|510143242|gb|A...|
|gi|510143242|gb|A...|
|gi|510143242|gb|A...|
|gi|510143242|gb|A...|
+--------------------+
only showing top 5 rows

None


+--------------------+
| Sequence_MD5_digest|
+--------------------+
|d6f8e49a4de47c68d...|
|d6f8e49a4de47c68d...|
|d6f8e49a4de47c68d...|
|d6f8e49a4de47c68d...|
|d6f8e49a4de47c68d...|
+--------------------+
only showing top 5 rows

None


...

+------------------------------+
|InterPro_annotations_accession|
+------------------------------+
|                     IPR020845|
|                     IPR020845|
|                     IPR020845|
|                     IPR020845|
|                     IPR020845|
+------------------------------+
only showing top 5 rows

None


+--------------------------------+
|InterPro_annotations_description|
+--------------------------------+
|            AMP-binding, cons...|
|            AMP-binding, cons...|
|            AMP-binding, cons...|
|            AMP-binding, cons...|
|            AMP-binding, cons...|
+--------------------------------+
only showing top 5 rows

None


+--------------+
|GO_annotations|
+--------------+
|             -|
|             -|
|             -|
|             -|
|             -|
+--------------+
only showing top 5 rows

None


+--------------------+
|Pathways_annotations|
+--------------------+
|MetaCyc: PWY-1061...|
|MetaCyc: PWY-1061...|
|MetaCyc: PWY-1061...|
|MetaCyc: PWY-1061...|
|MetaCyc: PWY-1061...|
+--------------------+
only showing top 5 rows

None


+--------------------+
|                perc|
+--------------------+
|0.001729831734549...|
|0.001729831734549...|
|0.001729831734549...|
|0.001729831734549...|
|0.001729831734549...|
+--------------------+
only showing top 5 rows

None


### modeling

In [12]:
# Organize data into labels for easier use
label_names = ["InterPro_annotations_accession"]
labels = IPRO_filt.select(label_names)
feature_names = [i for i in IPRO_filt.columns if i not in label_names]
features = IPRO_filt.select(feature_names)

print(label_names)
print(labels)
print(feature_names)
print(features)

['InterPro_annotations_accession']
DataFrame[InterPro_annotations_accession: string]
['Protein_accession', 'Sequence_MD5_digest', 'Sequence_length', 'Analysis', 'Signature_accession', 'Signature_description', 'Start_location', 'Stop_location', 'Score', 'Status', 'Date', 'InterPro_annotations_description', 'GO_annotations', 'Pathways_annotations']
DataFrame[Protein_accession: string, Sequence_MD5_digest: string, Sequence_length: int, Analysis: string, Signature_accession: string, Signature_description: string, Start_location: int, Stop_location: int, Score: float, Status: string, Date: string, InterPro_annotations_description: string, GO_annotations: string, Pathways_annotations: string]


In [ ]:
# get a list of models to evaluate

def get_models():
	models = {'lr': LogisticRegression()}
	models['knn'] = KNeighborsClassifier()
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	return models
 
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
	return cross_val_score(
	    model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
 
# define dataset
X = train
y = train_labels

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('model %s accuracy: %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# model evaluation
bayes = GaussianNB()
model = bayes.fit(train, train_labels)
preds = bayes.predict(test)

print(f"accuracy: {accuracy_score(test_labels, preds)}")
cf_matrix = confusion_matrix(test_labels, preds)
# print(confusion_matrix)
# turn confusion matrix into percentages
df_cm = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis] 
plt.figure()
heatmap = sns.heatmap(df_cm, cmap="Blues", annot=True)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Naive base Model Results in confusion matrix')
plt.show()  

print(classification_report(test_labels, preds))

In [ ]:
# closing the spark sessison

print('Closing spark session')
spk.sparkContext.stop()

In [11]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask_ml.preprocessing import OneHotEncoder
from dask_ml import model_selection
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from dask.distributed import Client
from dask_ml.wrappers import ParallelPostFit
import joblib
import time
import pickle
from scipy.sparse import csr_matrix


# get the start time
st = time.time()
 
path='/data/dataprocessing/interproscan/all_bacilli.tsv'
# path='/students/2021-2022/master/DaanSteur_DSLS/part.tsv' # 1million lines

def read_clean(path):
    """ Function to read the data and some data processing
    Arguments:
        path: path where the file is located as string
       
    Return:
        ddf: dask dataframe
    """
    ddf = dd.read_csv(path,sep='\t',usecols=[0,2,6,7,11],header=None,blocksize="50MB")

    #Renaming the columns
    ddf=ddf.rename(columns={0: "Protein", 2:'Sequence_length', 6:'Start', 7:'Stop', 11:'Interpro_accession',}) 
    
    #deleting the null interpro accessions
    ddf=ddf.loc[~(ddf['Interpro_accession']=='-')]

    #dropping duplicates
    ddf=ddf.drop_duplicates()

    #creating feat length column
    ddf['feat_len']=(ddf.Stop-ddf.Start)/ddf['Sequence_length']
    return ddf

def fun1(x):
    """ Function to check whether an Interpro accession is large or small.Used with apply method
    Arguments:
        x: dask dataframe  
    Return:
        0: if Interpro accession is small
        1: if Interpro accession is large
    """
    return 1 if x['feat_len']>.90 else 0


def find_size(x):
    """ Function that returns proteins with atleast one small and large interpro accession.
        Used with groupby apply method
    Arguments:
        x:groupby object grouped on protein
    Return:
        x:Returns the datframe
    """
    m=np.mean(x['size'])
    
    if (m>0) and (m<1) : 
        return x

def final_df(s_pivot,l_df):
    """ Function to merge large dataframe with small dataframe on protein.
    Arguments:
        s_pivot: Pivoted dataframe in which columns are the name of small interpro accession 
                 and values are their counts
        l_df:  Dask datframe containing the largest interpro accession of a protein
    Return:
        f_df: Final dataframe for machine learning
    """
    f_df=s_pivot.merge(l_df,how='inner',left_on='Protein',right_on='Protein')
    f_df=f_df.drop(columns=['feat_len'])
    f_df=f_df.replace(np.nan, 0)
    f_df=f_df.reset_index()
    return f_df

## Machine learning

def ml_dfs(f_df):
    """ Function to perform one hot encoding on class labels and to do train test splitting
    Arguments:
        f_df: Dask dataframe with the count of small interpro accession and 
              the name of largest interpro accession of each protein
    Return:
        X_train,y_train: Training dask arrays
        X_test,y_test: Testing dask arrays 
    """
    y = OneHotEncoder().fit_transform(f_df[['Interpro_accession']])
    X=f_df.iloc[:,2:-1].to_dask_array(lengths=True)
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2,random_state=42,convert_mixed_types=True)

    return X_train, X_test, y_train, y_test


if __name__ == "__main__":
    ddf=read_clean(path)

    ddf['size']=ddf.apply(lambda x:fun1(x),axis=1)

    #creating dataframe with proteins which have atleast one big and small interpro accession
    full_df=ddf.groupby(['Protein',]).apply(find_size,meta={'Protein':'str','Sequence_length':'int64','Start':'int64',
'Stop':'int64','Interpro_accession':'str','feat_len':'int64','size':'int64',}).reset_index(drop=True)
    
    #splittng dataframe into two containing large and small interpro accessions
    large_df=full_df.loc[full_df['size']==1]
    small_df=full_df.loc[full_df['size']==0]

    client = Client()
    with joblib.parallel_backend('dask'):

        #Finding the largest interpro accession of a protein
        l_df = large_df.loc[large_df.groupby(['Protein'])['feat_len'].transform(max) == large_df['feat_len'],['Protein','Interpro_accession','feat_len',]]
        #counting the number of each small interpro accession#transform('count')#agg('count').reset_index()
        s_df=small_df.groupby(['Protein','Interpro_accession'])['size'].agg('count').reset_index()

        #catego
        s_df=s_df.categorize(columns=['Interpro_accession'])
        l_df=l_df.categorize(columns=['Interpro_accession'])

        s_pivot=dd.reshape.pivot_table(s_df,index='Protein',columns='Interpro_accession', values='size')

        f_df=final_df(s_pivot,l_df)

        X_train, X_test, y_train, y_test=ml_dfs(f_df)

    mid_time=(time.time() - st)/60
    print('Entering ML part', mid_time, 'minutes')


    with joblib.parallel_backend('dask'):
        # creating randomforestclassifier object
        clf = RandomForestClassifier(random_state=0)
        clf.fit(X_train, y_train)
        # performing predictions on the test dataset
        y_pred = clf.predict(X_test)
        # using metrics module for accuracy calculation
        accuracy=metrics.accuracy_score(y_test, y_pred)
        
        filename = '/students/2021-2022/master/programming3/randforest_model.pkl'
        pickle.dump(clf, open(filename, 'wb'))
        
        X_df=dd.from_dask_array(X_train,columns=f_df.columns[2:-1])

        X_df.to_csv('/students/2021-2022/master/DaanSteur_DSLS/X_train.csv')

        print("ACCURACY OF THE MODEL: ", accuracy)
        
    # get the end time
    et = time.time()

    # get the execution time
    elapsed_time = (et - st)/60
    print('Execution time:', elapsed_time, 'minutes')

AttributeError: module 'pandas' has no attribute 'Int64Index'